# Time Weighted VectorStore Retriever

This retriever uses a combination of semantic similarity and recency.

The algorithm for combining them is basically:

```
semantic_similarity + decay_factor ** hours_passed
```

Notably, hours_passed refers to the hours passed since the object in the retriever was last accessed, not since it ws created.

In [1]:
from langchain.retrievers import TimeWeightedVectorStoreRetriever
import faiss
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
import time

## High decay factor

With a relatively high decay factor, this will mostly return documents that are semantically similar

In [2]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})
retriever = TimeWeightedVectorStoreRetriever(vectorstore=vectorstore, decay_factor=.99, k=1) 

In [3]:
retriever.add_documents([Document(page_content="hello world")])
time.sleep(1)
retriever.add_documents([Document(page_content="hello foo")])

['73f8f585-9536-4240-aef7-cea205b336bd']

In [4]:
retriever.get_relevant_documents("hello world")

ValueError: normalize_score_fn must be provided to FAISS constructor to normalize scores

## Low decay factor
With a low decay factor (in this, to be extreme, we will set close to 0) this will return most recent docs

In [30]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})
retriever = TimeWeightedVectorStoreRetriever(vectorstore=vectorstore, decay_factor=.00000001, k=1) 

In [31]:
retriever.add_documents([Document(page_content="hello world")])
time.sleep(1)
retriever.add_documents([Document(page_content="hello foo")])

['3c156c9a-ad6b-47dc-befc-87437a603973']

In [32]:
retriever.get_relevant_documents("hello world")

0.8416762384156714
1.6166791948060522
0.5781175231321124
1.577325318264064


[Document(page_content='hello foo', metadata={'last_accessed_at': datetime.datetime(2023, 4, 13, 23, 35, 54, 905605), 'created_at': datetime.datetime(2023, 4, 13, 23, 35, 54, 242988), 'buffer_idx': 1})]